Обучение без учителя преследует за собой две основные задачи:

- сжатие размерности

- кластеризация

# Сжатие размерности

## PCA - метод главных компонент

### Сингулярное разложение


*Сингулярным разложением* матрицы $X$ называется представление её в виде $X = U\Sigma V^T$, где:

 - $\Sigma$ есть $m\times n$ матрица у которой элементы, лежащие на главной диагонали, неотрицательны, а все остальные элементы равны нулю.
 - $U$ и $V$ – ортогональные матрицы порядка $m$  и $n$ соответственно.

Напомним, что ортогональная матрица - это матрица, для которой выполняется: $$A A^T = A^T A = E$$
 
Рассмотрим более подробно задачу о сингулярном разложении матрицы $X \in \mathbb{R}^{m \times n}$.
 
Элементы главной диагонали матрицы $D$ называются *сингулярными числами* матрицы $X$, а столбцы $U$ и $V$ левыми и правыми *сингулярными векторами* матрицы $X$.

Заметим, что матрицы $XX^T$ и $X^TX$ являются симметрическими неотрицательно определенными матрицами, и поэтому ортогональным преобразованием могут быть приведены к диагональному виду, причем на диагонали будут стоять неотрицательные собственные значения этих матриц.

В силу указанных выше свойств матриц $X^TX$ и $XX^T$ сингулярное разложение матрицы $X$ тесно связано с задачей о спектральном разложении этих матриц. Более точно:
- Левые сингулярные векторы матрицы $X$ (составляют матрицу U)– это собственные векторы матрицы $XX^T$.
- Правые сингулярные векторы матрицы $X$ (составляют матрицу V) – это собственные векторы матрицы $X^TX$.
- Сингулярные числа матрицы $X$ - это корни из собственных значений матрицы $X^TX$ (или $XX^T$).

Таким образом, для нахождения сингулярного разложения матрицы $X$ необходимо, найти собственные векторы и значения матриц $X^TX$ и $XX^T$
и составить из них матрицы $U, V, \Sigma$.

![Графическая интерпретация сингулярного уравнения](https://miro.medium.com/max/700/1*6wkgGgBy2NLVmRVOw8K86w.png)


In [ ]:
import numpy as np
from scipy.linalg import svd
# define a matrix
X = np.array([[1, 2], [3, 4], [5, 6]])
print(X)
# SVD
U, s, VT = svd(X)
print(U)
print(s)
print(VT)

In [ ]:
# Проверим написанные нами выше формулы
w_left, v_left = np.linalg.eig(X @ X.T)
print(v_left)  # - матрица U
w_right, v_right = np.linalg.eig(X.T @ X)
print(v_right.T)


### Идея и реализация метода PCA

Основная идея метода главных компонент заключается в нахождении таких попарно ортогональных направлений в исходном многомерном пространстве (признаковом), вдоль которых данные имеют наибольший разброс (выборочную дисперсию). Эти направления называются главными компонентами. 

Другая формулировка PCA – для данной многомерной случайной величины построить такое ортогональное преобразование координат, что в результате корреляции между отдельными координатами обратятся в ноль. Таким образом, задача сводится к диагонализации матрицы ковариаций, что эквивалентно нахождению сингулярного разложения матрицы исходных данных.

**Алгоритм**:

1. Определить $k<n$ – новую размерность (k - компонент, n - исходное количество признаков)
2. Нормализовать данные:
 - Вычесть из $X$ среднее, то есть заменить все $\Large x^{(i)}$  на $$\Large  x^{(i)} - \frac{1}{m} \sum_{i=1}^{m}{x^{(i)}}$$
 - Привести данные к единичной дисперсии: посчитать $$\Large  \sigma_j^2 = \frac{1}{m} \sum_{i=1}^{m}{(x^{(i)})^2}$$
и заменить $\Large x_j^{(i)}$ на $\Large \frac{x_j^{(i)}}{\sigma_j}$ 
3. Найти сингулярное разложение матрицы $X$:
$$\Large X = UDV^T$$
4. Положить $V =$ [$k$ левых столбцов матрицы $V$]
5. Вернуть новую матрицу $$\Large Z = XV \in \mathbb{R}^{m \times k}$$

### Пример реализации

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

np.random.seed(0)
mean = np.array([0.0, 0.0])
cov = np.array([[1.0, -1.0], 
                [-2.0, 3.0]])
X = np.random.multivariate_normal(mean, cov, 300)

pca = PCA()
X_scal = (X - X.mean(axis=0))/X.std(axis=0)
pca.fit(X_scal)
print('Proportion of variance explained by each component:\n' +\
      '1st component - %.2f,\n2nd component - %.2f\n' %
      tuple(pca.explained_variance_ratio_))
print('Directions of principal components:\n' +\
      '1st component:', pca.components_[0],
      '\n2nd component:', pca.components_[1])

plt.figure(figsize=(10,10))
plt.scatter(X[:, 0], X[:, 1], s=50, c='r')
for l, v in zip(pca.explained_variance_ratio_, pca.components_):
    d = 5 * np.sqrt(l) * v
    plt.plot([0, d[0]], [0, d[1]], '-k', lw=3)
plt.axis('equal')
plt.title('2D normal distribution sample and its principal components')
plt.show()

Первая главная компонента (ей соответствует более длинный вектор) объясняет более 90% дисперсии исходных данных. Это говорит о том, что она содержит в себе почти всю информацию о расположении выборки в пространстве, и вторая компонента может быть опущена. Спроецируем данные на первую компоненту.

In [ ]:
# Реализация через SVD

U, s, Vh = svd(X_scal, full_matrices=False)
n_comp = 2
Z1 = U[:, :n_comp] * s[:n_comp]
Z2 = X_scal @ Vh.T
Z1[:10]
print(np.allclose(Z1, Z2))

In [ ]:
pca = PCA()
pca.fit(X_scal)
pca.transform(X_scal)[:10]

Заметим, что знаки не совпадают. SVD страдает от проблемы, называемой "неопределенностью знака",
которая означает, что знак компонент и преобразованные данные зависят от алгоритма и случайного состояния. Для решения
этой проблемы мы можем раскладывать не по всем векторам, а по зараннее выбранным (например, как в PCA - выбрать
зараннее известное количество компонент): $ X_k = \sum_{i=1}^k \sigma_i u_i v_i $ вместо
$ X_n = \sum_{i=1}^n \sigma_i u_i v_i $. Этот метод называется усеченным SVD (truncated SVD).



In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_trunc = TruncatedSVD(n_components=1)
svd_trunc.fit(X_scal)
result = svd_trunc.transform(X_scal)
print(result[:10])


In [ ]:
# Оставим столько компонент, сколько нужно для объяснения по крайней мере 90% вариации
pca = PCA(0.90)
X_reduced = pca.fit_transform(X)

# Преобразуем сниженные данные в изначальное признаковое пространство
X_new = pca.inverse_transform(X_reduced)

plt.figure(figsize=(10,10))
plt.plot(X[:, 0], X[:, 1], 'or', alpha=0.3)
plt.plot(X_new[:, 0], X_new[:, 1], 'or', alpha=0.8)
plt.axis('equal')
plt.title('Проекция всей выборки на первую компоненту')
plt.show()

Мы понизили размерность данных вдвое, при этом сохранив наиболее значимые черты.
В этом заключается основной принцип понижения размерности – приблизить многомерный набор данных
с помощью данных меньшей размерности, сохранив при этом как можно больше информации об исходных данных.

## PCR - Principal component regression (регрессия на главные компоненты)

PCR обычно используют в ситуациях, когда число факторов велико, а число наблюдений мало. В этом случае число степеней
свободы будет стремиться к нулю, что плохо скажется на результатах. Помимо этого, PCR хорошо заходит в моделях,
где факторы сильно коррелируют друг с другом.

Пусть $Z_m$ - векторы новой размерности $m$, которые представлены через линейную комбинацию исходных векторов:  
$$Z_m = \sum_{j=1}^p \phi_{jm}X_j$$

где $\phi_{jm}$ - вес фактора $j$ в компоненту $m$ согласно методу PCA

Тогда после вычисления МНК-оценок данных векторов на целевую переменную, коэффициенты для исходных факторов находятся по такой формуле:

$$\beta_j = \sum_{m=1}^M \theta_m \phi_{jm}$$

где $\theta_m$ - МНК оценка компоненты $m$ в линейной регрессии.

Алгоритм метода PCR:

1. Находим $m$ главных компонент из признакового пространства размерности $p$

2. Строим регрессию зависимой переменной на эти главные компоненты.

3. Полученные коэффициенты матрично перемножаем с весами, с которыми каждая исходная переменная входит в компоненту.

4. Интерпретируем результаты. Поскольку все переменные стандартизированные, веса коэффициентов можно сравнивать друг с другом.

Количество компонент в данном случае - гиперпараметр, который мы настраиваем при помощи кросс-валидации.

In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import category_encoders as ce
import statsmodels.api as sm


credit = pd.read_csv("/Users/iakubovskii/Machine_Learning/RANEPA/Fintech_2020/Машинное обучение/Данные/Credit.csv")
credit.set_index("ID", inplace=True)

qual_cols = ['Gender', 'Student', 'Married', 'Ethnicity']
credit_reg = credit.drop(qual_cols, axis=1).join(pd.get_dummies(credit[qual_cols], drop_first=True))
X_reg, y_reg = credit_reg.drop("Balance", axis=1), credit_reg['Balance']
sc = StandardScaler()
X_reg_scal, y_reg_scal = sc.fit_transform(X_reg), sc.fit_transform(y_reg.values.reshape(-1,1))
n_pca=2
pca = PCA(n_components=n_pca).fit(X_reg_scal)
pcs = pca.fit_transform(X_reg_scal)
X_PCR =  pcs.copy()
V = pca.components_.T
ols = sm.OLS(endog = y_reg_scal, exog = sm.add_constant(X_PCR)).fit()
print(ols.summary())
beta_Z = ols.params[1:]
beta_X = V @ beta_Z
coefs = pd.DataFrame(index = X_reg.columns, data = beta_X)
coefs.columns = ['coefs_PCR']

In [ ]:
categorical_features = ['Gender', 'Student', 'Married', 'Ethnicity']
numeric_features = list(set(credit.columns) - set(categorical_features) - set(["Balance"]))

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_std=True, with_mean=True))])

categorical_transformer = ce.OneHotEncoder()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
X_preprocessed = preprocessor.fit_transform(credit.drop("Balance", axis=1))
y = (credit['Balance'] - credit['Balance'].mean()) / credit['Balance'].std

n_pca=2
pca = PCA(n_components=n_pca).fit(X_preprocessed)
pcs = pca.fit_transform(X_preprocessed)
X_PCR =  pcs.copy()
V = pca.components_.T
ols = sm.OLS(endog = y, exog = sm.add_constant(X_PCR)).fit()
print(ols.summary(xname=['const', 'PC1', "PC2"]))
beta_Z = ols.params[1:]
beta_X = V @ beta_Z
all_cols = preprocessor.transformers_[0][2].copy()  # extract num columns (0 step in pipeline)
cat_col_names = preprocessor.transformers_[1][1].get_feature_names()  # extract category columns (1 step in pipeline)
all_cols.extend(cat_col_names)
coefs = pd.DataFrame(index = all_cols, data = beta_X)
coefs.columns = ['coefs_PCR']
print(coefs)


Почему коэффициенты отличаются? Ваша задача с этим разобраться. А мы пока посмотрим, как снижается MSE при увеличении
количества компонент в регрессии, а также как изменяются сами коэффициенты.

In [ ]:
def get_mse(n_pca=2):
    pca = PCA(n_components=n_pca).fit(X_reg_scal)
    pcs = pca.fit_transform(X_reg_scal)
    X_PCR =  pcs.copy()
    V = pca.components_.T
    ols = sm.OLS(endog = y_reg_scal, exog = sm.add_constant(X_PCR)).fit()
    return ols.mse_model
plt.plot(np.arange(1, 11),list(map(get_mse, np.arange(1, 11))));

In [ ]:
def get_coefs(n_pca=2):
    pca = PCA(n_components=n_pca).fit(X_reg_scal)
    pcs = pca.fit_transform(X_reg_scal)
    X_PCR =  pcs.copy()
    V = pca.components_.T
    ols = sm.OLS(endog = y_reg_scal, exog = sm.add_constant(X_PCR)).fit()
    beta_Z = ols.params[1:]
    beta_X = V @ beta_Z
    coefs = pd.DataFrame(index = X_reg.columns, data = beta_X)
    coefs.columns = ['coefs_PCR_' + str(n_pca)]
    return coefs
get_coefs(3)
from functools import reduce
dfs = [get_coefs(i) for i in range(1, 11)]
df_final = reduce(lambda left,right: left.join(right), dfs)
df_final

## PLS - partial least squares (частный МНК)

PCR предполагает регрессию зависимой переменной на компоненты, которые получены из задачи обучения без учителя.
PLS, в свою очередь, определяет новые переменные, используя зависимость изначальных факторов с целевым показателем.
PLS помогает найти те направления, которые помогают объяснить как целевой фактор, так и объясняющие переменные.

Алгоритм PLS выглядит примерно следующим образом.

1. Первая компонента $Z_1$ находится по формуле $Z_1 = \sum_{j=1}^p corr_{j1}^2 X_j$. 

2. На втором шаге мы корректируем все факторы по $Z_1$ - строим регрессии каждой переменной на $Z_1$ и находим остатки. Как мы помним, остатки - это необъясненная доля дисперсии целевой переменной. В нашем случае это будет необъясненная доля дисперсии первой компонентой. 

3. Аналогично, как и на первом шаге, для каждой компоненты считаем веса, только вместо $X_1$ будут их остатки от объяснения предыдущих компонент.

4. Оцениваем зависимую переменную на полученные компоненты.

In [ ]:
# PLS
from sklearn.cross_decomposition import PLSRegression
pls = PLSRegression(n_components=2)
pls.fit(X_reg_scal, y_reg_scal)
print(f"PLS r-squared with 2 components: {pls.score(X_reg_scal, y_reg_scal):.3f}")


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
pca_2 = make_pipeline(PCA(n_components=2), LinearRegression())
pca_2.fit(X_reg_scal, y_reg_scal)
print(f"PCR r-squared with 2 components {pca_2.score(X_reg_scal, y_reg_scal):.3f}")
